In [ ]:
from aiida import load_profile
load_profile();
import ipywidgets as widgets
from IPython.display import display
from aiida.orm import QueryBuilder
from widgets import sens_query
import aiidalab_widgets_base as awb
from aiida import orm,plugins
from widgets import locations

NetCDF = plugins.DataFactory('netcdf.data')
w_inspect = plugins.WorkflowFactory('inspect.workflow')
cs = plugins.CalculationFactory("collect.sens")
base_address = '/project/s1152/shenne/PARIS/NAME_footprints/EUROPE/'

# Title

### Inspect

Base address: /project/s1152/shenne/PARIS/NAME_footprints/EUROPE/

In [ ]:
style = {"description_width": "initial"}

address = widgets.Text(
            value = 'LUT-60magl/inert',
            description="Remote address",
            style=style,
        )
def prepare_inspect():
    computer = orm.load_computer(label='daint')
    remote_path = orm.RemoteData(remote_path = base_address+address.value,
                             computer = computer)
    remote_path_2 = orm.RemoteData(remote_path = base_address+'OPE-120magl/inert',
                             computer = computer)
    builder = w_inspect.get_builder()
    builder.remotes = {'a':remote_path,}
                      #'b':remote_path_2}
    return builder

btn_submit_1 = awb.SubmitButtonWidget(
    w_inspect,
    inputs_generator = prepare_inspect,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_1.btn_submit.button_style="info"
btn_submit_1.btn_submit.description="Launch inspect"

line_h = widgets.HTML(
            value="""<hr>
            """
        )
info_text = widgets.HTML(
            value="""Find for nc lise in stash folder of previous 
            Collect sensitivities calclations.
            """
        )

def prepare_inspect_cs():
    qb = orm.QueryBuilder()
    qb.append(cs, tag="cs", filters={"attributes.exit_status": 0})
    qb.append(orm.RemoteStashFolderData, with_incoming="cs", project="*")
    builder = w_inspect.get_builder()
    print({f'test_{j}':i[0] for j,i in enumerate(qb.all())})
    directories_ = {f'test_{j}':i[0] for j,i in enumerate(qb.all())}
    if directories_:
        builder.remotes_cs = directories_
        return builder
    else:
        print('No files found.')



btn_submit_2 = awb.SubmitButtonWidget(
    w_inspect,
    inputs_generator = prepare_inspect_cs,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_2.btn_submit.button_style="success"
btn_submit_2.btn_submit.description="Launch CS inspect"

acc = widgets.Accordion(
            children=[
                 widgets.VBox(
                    children=[
                address,
                btn_submit_1,
                line_h,
                info_text,
                btn_submit_2
                    ])
            ],
        )
acc.set_title(0, "Inspect")
acc.selected_index = None
acc

###  Search

In [ ]:
sq = sens_query.SearchSens()
sq